# Segmentation de déchets avec YOLOv8 - TEKBOT Robotics Challenge 2025
La segmentation des instances va plus loin que la détection des objets et consiste à identifier des objets individuels dans une image et à les segmenter par rapport au reste de l'image.

Le résultat d'un modèle de segmentation d'instances est un ensemble de masques ou de contours qui délimitent chaque objet de l'image, ainsi que des étiquettes de classe et des scores de confiance pour chaque objet. La segmentation d'instances est utile lorsque vous avez besoin de savoir non seulement où se trouvent les objets dans une image, mais aussi quelle est leur forme exacte.

## Configuration et importation des bibliothèques

In [ ]:
!pip install ultralytics roboflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 88.8 MB/s eta 0:00:00


---

## 🗂️ Collecte et annotation des données

Pour entraîner un modèle de segmentation des instances sur des **images de déchets**, vous devez constituer un **jeu de données structuré** et bien annoté.

Commencez par réunir un ensemble représentatif d’**images de déchets** (photos prises sur le terrain, téléchargées ou issues de bases de données ouvertes). Chaque image devra ensuite être annotée avec des **masques polygonaux** représentant chaque type de déchet de manière individuelle.

Le jeu de données final doit respecter la structure suivante :

```
dataset/
├── images/
│   ├── train/    # Images utilisées pour l'entraînement
│   └── val/      # Images utilisées pour la validation
├── labels/
│   ├── train/    # Fichiers d’annotation des images d’entraînement (format YOLO + segmentation)
│   └── val/      # Fichiers d’annotation des images de validation
├── data.yaml     # Fichier de configuration décrivant le dataset et les classes
```

Le fichier `data.yaml` doit contenir :

```yaml
path: dataset
train: images/train
val: images/val
names:
  0: plastique
  1: métal
  2: papier
  3: verre
```

💡 **Conseil** : veillez à équilibrer les classes dans les images (ex : autant de plastique que de métal) pour éviter les biais lors de l'entraînement.


Voici un guide clair et structuré pour l'annotation des images :

1. **Annoter les images avec Roboflow (via navigateur)**
2. **Connecter ton projet Roboflow à Google Colab** pour l'entraînement avec YOLOv8 (Ultralytics)

---

## ✅ 1. Annoter les images avec Roboflow

### 🔧 Étapes :

1. **Créer un compte** sur [https://roboflow.com](https://roboflow.com) (gratuit).
2. Clique sur **"Create New Project"** :

   * Nom : `dechets-segmentation`
   * Type de projet : `Instance Segmentation`
   * Annotation group : `One annotation group`
   * License : choisir selon ton cas
3. Une fois le projet créé, clique sur **Upload** pour importer tes images :

   * Format accepté : `.jpg`, `.png`, etc.
   * Roboflow redimensionnera les images si nécessaire.
4. Clique sur **Annotate** :

   * Sélectionne une image
   * Clique sur **"Add Annotation"**
   * Trace un **polygone** autour de chaque déchet
   * Donne-lui un nom (ex: `plastique`, `papier`, etc.)
5. Quand tu as fini, clique sur **Save**.

---

## 🔄 2. Générer et exporter le dataset pour YOLOv8

1. Une fois toutes les images annotées :

   * Clique sur **"Generate Dataset"** en haut à droite
   * Choisis :

     * Format : **YOLOv8 Segmentation**
     * Taille d’image : 640x640 ou selon ton besoin
     * Data split : train / val / test
     * Procéder à l'augmentation éventuelle des données et/ou préprocessing

2. Clique sur **"Download Code"** > **Show download code**

   Tu obtiendras un **script d'import automatique** pour Colab, comme ceci :

```python
!pip install roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="TON_API_KEY")
project = rf.workspace("TON_WORKSPACE").project("dechets-segmentation")
dataset = project.version(1).download("yolov8")
```

---

## 🧪 3. Lier à ton notebook Google Colab

### 📘 Exemple complet dans Colab :

```python
# 1. Installer les dépendances
!pip install ultralytics roboflow

# 2. Télécharger les données Roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="TON_API_KEY")
project = rf.workspace("TON_WORKSPACE").project("dechets-segmentation")
dataset = project.version(1).download("yolov8")  # format YOLOv8 Segmentation

# 3. Entraîner avec YOLOv8
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")  # ou yolov8m-seg.pt, etc.
model.train(data=dataset.location + "/data.yaml", epochs=50, imgsz=640, task="segment")
```

---

# 📄 Vérifie que le fichier YAML est bien téléchargé
**!cat {dataset.location}/data.yaml**

## 📌 Où trouver les infos pour `api_key` et `workspace` ?

* Va sur [https://app.roboflow.com](https://app.roboflow.com)
* Dans ton tableau de bord, clique sur ton profil → **"Your API Key"**
* Le **nom du workspace** est visible dans l’URL : `https://app.roboflow.com/<workspace-name>/<project-name>`

---



In [ ]:
# 🧪 Charger ton modèle entraîné
model = YOLO("yolov8-segmentation-dechets/exp/weights/best.pt")

# 📸 Tester sur une image
results = model("https://images.unsplash.com/photo-1603252109303-2751441dd157")  # image en ligne
results[0].show()

* 📦 Export vers différents formats (`.pt`, `.onnx`, `.tflite`, `.torchscript`, etc.)
* 🤖 Déploiement sur **Raspberry Pi**
* 📡 Utilisation avec **Arduino + ESP32/ESP8266**
* 📱 Déploiement mobile (Edge, Android, iOS)

---

### 🔄 À ajouter après l'entraînement

```python
# 📤 Exportation du modèle YOLOv8 adapté à vos donées vers différents formats

# 📦 Charger ton modèle entraîné
model = YOLO("yolov8-segmentation-dechets/exp/weights/best.pt")

# 🔽 Exporter vers TorchScript (pour Raspberry Pi, C++ backend)
model.export(format="torchscript")

# 🔁 Exporter vers ONNX (compatible OpenCV DNN, TensorRT, etc.)
model.export(format="onnx")

# 📱 Exporter vers CoreML (pour iOS)
model.export(format="coreml")

# 🤖 Exporter vers TensorFlow Lite (Edge / Android / ESP)
model.export(format="tflite")

# 🧠 Exporter vers TensorFlow SavedModel (ex : pour TensorFlow.js, etc.)
model.export(format="saved_model")

# 📡 Export vers PaddlePaddle (optionnel)
model.export(format="paddle")
```

---

### 🍓 Utilisation sur **Raspberry Pi** (avec TorchScript ou ONNX)

```bash
# Sur Raspberry Pi (une fois le modèle exporté) :
# Installer les dépendances
sudo apt install libopencv-dev python3-opencv
pip install torch torchvision ultralytics

# Exemple de script Python minimal
import torch
from PIL import Image
import cv2
import numpy as np

# Charger modèle TorchScript
model = torch.jit.load("best.torchscript")
model.eval()

# Lire une image
img = cv2.imread("image.jpg")
img_resized = cv2.resize(img, (640, 640))
img_tensor = torch.tensor(img_resized / 255.0).permute(2, 0, 1).unsqueeze(0).float()

# Prédire
with torch.no_grad():
    pred = model(img_tensor)
```

---

### 📡 Utilisation avec **Arduino + ESP32/ESP8266**

> ⚠️ YOLOv8 ne s’exécute pas directement sur un ESP32 ou un Arduino. Mais tu peux :

* Exécuter YOLOv8 sur un **serveur (Raspberry, cloud)** et
* Envoyer les **résultats via HTTP/MQTT** à l’ESP32/ESP8266 pour action (LED, moteur, etc.)

#### Exemple d’architecture :

```text
Camera -> Raspberry Pi (YOLOv8) -> Envoi JSON vers ESP32 -> ESP réagit (alarme, LED, etc.)
```

#### Exemple de code Python serveur :

```python
import requests
import json

results = model("photo.jpg")
labels = results[0].boxes.cls.cpu().numpy().tolist()

# Envoyer à l’ESP
requests.post("http://192.168.0.100:5000/detect", json={"labels": labels})
```

#### Code côté ESP (MicroPython ou Arduino C++) :

```cpp
// Côté Arduino ou ESP32 - lire JSON depuis HTTP/MQTT et agir
if (received_label == "plastique") {
  digitalWrite(LED_PIN, HIGH);
}
```

---

### 📱 Déploiement mobile (Android/iOS avec TensorFlow Lite)

* Utilise le fichier `best.tflite`
* Intègre-le dans une app Android via **TensorFlow Lite Android API**
* Exemples disponibles ici : [https://www.tensorflow.org/lite/android](https://www.tensorflow.org/lite/android)

```java
// Android Java
Interpreter tflite = new Interpreter(loadModelFile("best.tflite"));
```

---

### ✅ Récapitulatif des formats utiles

| Format         | Utilisation                         |
| -------------- | ----------------------------------- |
| `.pt`          | PyTorch / Ultralytics               |
| `.torchscript` | Raspberry Pi / C++ / PyTorch Lite   |
| `.onnx`        | OpenCV DNN, TensorRT, NVIDIA Jetson |
| `.tflite`      | Android, Raspberry Pi, ESP-CAM      |
| `coreml`       | iOS (iPhone, iPad)                  |
| `saved_model`  | TensorFlow, TF.js, TPU              |

---




---
## Documentation complémentaire

### Roboflow : https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://roboflow.com/&ved=2ahUKEwj4xKbqtOSNAxXCTaQEHZpOM5QQFnoECAkQAQ&usg=AOvVaw1h_eItJcl0qix45QGoUODw

### Tensorflow : https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.tensorflow.org/%3Fhl%3Dfr&ved=2ahUKEwiz25OGteSNAxW1V6QEHQs2AyUQFnoECBUQAQ&usg=AOvVaw1C86YCtph6fCCJ-ya1LX3g

### Pytorch : https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://pytorch.org/&ved=2ahUKEwiQzqOSteSNAxUXV6QEHTirDjMQFnoECBkQAQ&usg=AOvVaw2mABY6VbqZdRJYnleMzDSb

### Ultralytics : https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.ultralytics.com/&ved=2ahUKEwih6cebteSNAxW8U6QEHSOwPP4QFnoECEEQAQ&usg=AOvVaw0egcjbygLKWOBkY23mDV-l
